# Viz + QG Data

In [ ]:
import os
import sys

from pyprojroot import here

# spyder up to find the root
root = here(project_files=[".root"])


# append to path
sys.path.append(str(root))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr

sns.reset_defaults()
sns.set_context(context="talk", font_scale=0.7)

import hvplot.pandas
import hvplot.xarray
from inr4ssh._src.data.ssh_obs import load_ssh_altimetry_data_train
from inr4ssh._src.features.temporal import MinMaxFixedScaler
from inr4ssh._src.operators.finite_diff import calculate_gradient, calculate_laplacian
from inr4ssh._src.preprocess.coords import (
    correct_coordinate_labels,
    correct_longitude_domain,
)
from inr4ssh._src.preprocess.missing import (
    generate_random_missing_data,
    generate_skipped_missing_data,
)
from inr4ssh._src.preprocess.obs import add_noise
from inr4ssh._src.preprocess.subset import spatial_subset, temporal_subset
from inr4ssh._src.viz.movie import create_movie

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Observations

In [ ]:
!pwd

In [ ]:
!ls /Users/eman/code_projects/torchqg/data/

In [ ]:
train_data_dir = f"/Users/eman/code_projects/torchqg/data/qgsim_simple_128x128.zarr"
# train_data_dir = f"/Users/eman/.CMVolumes/cal1_workdir/data/qg_data/public/qg_sim.nc"
# train_data_dir =

ds_obs = xr.open_zarr(train_data_dir)

ds_obs

**Comments**:

* Case - Inspired from the paper (use the exact same parameters) - Gram Ringler
    * Graham and Ringler 2013 : https://doi.org/10.1016/j.ocemod.2013.01.004
    * Dimensional Version in the paper (Hugo may not be using that...)
* Non-dimensional version
* PyQG
* Later could be used for a data challenge :)

In [ ]:
# ds_obs.p.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

## Preprocessing

In [ ]:
def reformat_time(ds, dt):

    ds = ds - ds[0]
    ds /= dt

    return ds


def preprocessing(ds, time_min: int, time_max: int, dt: float = 1.0):

    # slice timesteps
    ds = ds.isel(steps=slice(time_min, time_max))

    # reformat time
    time_coords = ds.steps.values.astype(np.float64)

    time_coords = reformat_time(time_coords, dt)
    time_coords = time_coords.astype(np.float64)
    ds["steps"] = time_coords

    return ds

In [ ]:
ds_obs = preprocessing(ds_obs, 500, 500 + 30)

In [ ]:
ds_obs

In [ ]:
ds_obs.p.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

In [ ]:
# fig, ax = plt.subplots()

# ds_obs.p.plot.hist(bins=100, ax=ax, density=True)

# ax.set(
#     xlabel="",
#     ylabel="",
# )
# plt.tight_layout()
# plt.show()

## Observations

* Random Sampling
* Different Spatial Resolutions
* Different Temporal Resolutions
* Different Spatio-Temporal Resolutions
* Simulated Altimetry Tracks

### Random Sampling

#### 50% Coverage

In [ ]:
p_rand = generate_random_missing_data(
    ds_obs.p.values.copy(), missing_data_rate=0.50, return_mask=False, seed=123
)

ds_obs["obs_p_rand"] = (ds_obs.p.dims, p_rand)

In [ ]:
# ds_obs

In [ ]:
# ds_obs.obs_p_rand.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

#### 20% Coverage

In [ ]:
p_rand = generate_random_missing_data(
    ds_obs.p.values.copy(), missing_data_rate=0.80, return_mask=False, seed=123
)

ds_obs["obs_p_rand"] = (ds_obs.p.dims, p_rand)

In [ ]:
# ds_obs.obs_p_rand.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

#### 5% Coverage

In [ ]:
p_rand = generate_random_missing_data(
    ds_obs.p.values.copy(), missing_data_rate=0.95, return_mask=False, seed=123
)

ds_obs["obs_p_rand"] = (ds_obs.p.dims, p_rand)

In [ ]:
# ds_obs.obs_p_rand.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

#### 1% Coverage

In [ ]:
p_rand = generate_random_missing_data(
    ds_obs.p.values.copy(), missing_data_rate=0.99, return_mask=False, seed=123
)

ds_obs["obs_p_rand"] = (ds_obs.p.dims, p_rand)

In [ ]:
# ds_obs.obs_p_rand.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

In [ ]:
# fig, ax = plt.subplots()

# ds_obs.p.plot.hist(bins=100, ax=ax, density=True, label="True")
# ds_obs.obs_p_rand.plot.hist(bins=100, ax=ax, density=True, label="Observations")

# ax.set(
#     xlabel="",
#     ylabel="",
# )
# plt.tight_layout()
# plt.legend()
# plt.show()

### Spatial Resolution

In [ ]:
step = 3
data = generate_skipped_missing_data(ds_obs.p.values.copy(), step=step, dim=1)
data = generate_skipped_missing_data(data, step=step, dim=2)

In [ ]:
ds_obs["obs_p_spares"] = (ds_obs.p.dims, data)

In [ ]:
# ds_obs.obs_p_spares.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

### Temporal Resolution

In [ ]:
step = 3
data = generate_skipped_missing_data(ds_obs.p.values.copy(), step=step, dim=0)

ds_obs["obs_p_tempres"] = (ds_obs.p.dims, data)

In [ ]:
# ds_obs.obs_p_tempres.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

### Spatio-Temporal Resolution

In [ ]:
step = 5
data = generate_skipped_missing_data(ds_obs.p.values.copy(), step=step, dim=1)
data = generate_skipped_missing_data(data, step=step, dim=2)

step = 2
data = generate_skipped_missing_data(data, step=step, dim=0)

ds_obs["obs_p_spatempres"] = (ds_obs.p.dims, data)

In [ ]:
# ds_obs.obs_p_spatempres.hvplot.image(
#     x="Nx", y="Ny", width=500, height=400, cmap="viridis"
# )

### AlongTrack Simulations

In [ ]:
from inr4ssh._src.preprocess.obs import get_streaming_lines, add_obs_tracks

In [ ]:
rate = 5
axis = "y"
width = 2

# observations for the stream function
y_obs, _ = get_streaming_lines(
    ds_obs.p.values.copy(), rate=rate, axis=axis, width=width
)

ds_obs["obs_p_nadir"] = (ds_obs.p.dims, y_obs)

In [ ]:
ds_obs.obs_p_nadir.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

### Multiple Nadir

In [ ]:
rate = 5
axis = "y"
width = 2

# observations for the stream function
y_obs_nadir_1, _ = get_streaming_lines(
    ds_obs.p.values.copy(), rate=rate, axis=axis, width=width
)

rate = 5
axis = "y"
width = 2

y_obs_nadir_2, _ = get_streaming_lines(
    ds_obs.p.values.copy(), rate=rate, axis=axis, width=width, init=-60
)

# y_obs_nadirs = add_obs_tracks(y_obs_nadir_1, y_obs_nadir_2)

In [ ]:
y_obs_nadirs = np.ones_like(y_obs_nadir_1)
nt, nx, ny = y_obs_nadir_2.shape

for i in range(nt):
    for j in range(nx):
        for k in range(ny):
            a = y_obs_nadir_1[i, j, k]
            b = y_obs_nadir_2[i, j, k]

            if np.isnan(a) and np.isnan(b):
                y_obs_nadirs[i, j, k] = np.nan
            elif np.isnan(a) and not np.isnan(b):
                y_obs_nadirs[i, j, k] = b
            elif not np.isnan(a) and np.isnan(b):
                y_obs_nadirs[i, j, k] = a
            elif not np.isnan(a) and not np.isnan(b):
                y_obs_nadirs[i, j, k] = 0.5 * (a + b)
            else:
                raise ValueError(f"Something is wrong")

print(np.nan_to_num(y_obs_nadirs).min(), np.nan_to_num(y_obs_nadirs.max()))
y_obs_nadirs[y_obs_nadirs == 0] = np.nan

In [ ]:
ds_obs["obs_p_nadirs"] = (ds_obs.p.dims, y_obs_nadirs)

In [ ]:
ds_obs.obs_p_nadirs.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

### SWOT Tracks

In [ ]:
rate = 5
axis = "y"
width = 10

# observations for the stream function
y_obs_swot, _ = get_streaming_lines(
    ds_obs.p.values.copy(), rate=rate, axis=axis, width=width, init=+30
)

In [ ]:
ds_obs["obs_p_swot"] = (ds_obs.p.dims, y_obs_swot)

In [ ]:
ds_obs.obs_p_swot.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

In [ ]:
y_obs_swot_nadir = np.ones_like(y_obs_nadirs)
nt, nx, ny = y_obs_swot.shape

for i in range(nt):
    for j in range(nx):
        for k in range(ny):
            a = y_obs_nadirs[i, j, k]
            b = y_obs_swot[i, j, k]

            if np.isnan(a) and np.isnan(b):
                y_obs_swot_nadir[i, j, k] = np.nan
            elif np.isnan(a) and not np.isnan(b):
                y_obs_swot_nadir[i, j, k] = b
            elif not np.isnan(a) and np.isnan(b):
                y_obs_swot_nadir[i, j, k] = a
            elif not np.isnan(a) and not np.isnan(b):
                y_obs_swot_nadir[i, j, k] = 0.5 * (a + b)
            else:
                raise ValueError(f"Something is wrong")

print(np.nan_to_num(y_obs_swot_nadir).min(), np.nan_to_num(y_obs_swot_nadir.max()))
y_obs_swot_nadir[y_obs_swot_nadir == 0] = np.nan

In [ ]:
ds_obs["obs_p_swot_nadir"] = (ds_obs.p.dims, y_obs_swot_nadir)

In [ ]:
ds_obs.obs_p_swot_nadir.hvplot.image(
    x="Nx", y="Ny", width=500, height=400, cmap="viridis"
)

In [ ]:
ds_obs.to_netcdf(f"/Users/eman/code_projects/torchqg/data/qgsim_simple_128x128_obs.nc")

In [ ]:
ds_obs.obs_p_swot.plot.imshow(
    col="steps",
    robust=True,
    col_wrap=3,
    cmap="viridis",
)

In [ ]:
df_obs.dropna().shape, df_obs.shape

In [ ]:
# fig, ax = plt.subplots()

# ds_obs.obs_p.plot.hist(bins=100, ax=ax, density=True)

# ax.set(
#     xlabel="",
#     ylabel="",
# )
# plt.tight_layout()
# plt.show()

In [ ]:
# fig, ax = plt.subplots()

# ds_obs.p.plot.hist(bins=100, ax=ax, density=True, label="True")
# ds_obs.obs_p.plot.hist(bins=100, ax=ax, density=True, label="Observations")

# ax.set(
#     xlabel="Potential Vorticity",
#     ylabel="Density",
# )
# plt.tight_layout()
# plt.legend()
# plt.show()

In [ ]:
sigma = 0.1
noise = "gauss"
ds_obs["obs_p_n"] = add_noise(ds_obs.obs_p, sigma=sigma, noise=noise)

In [ ]:
fig, ax = plt.subplots()

ds_obs.obs_p_n.plot.hist(bins=100, ax=ax, density=True)
ds_obs.obs_p.plot.hist(bins=100, ax=ax, density=True)

ax.set(
    xlabel="",
    ylabel="",
)
plt.tight_layout()
plt.show()

In [ ]:
# ds_obs.obs_p_n.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

In [ ]:
# save_path = "./"
# create_movie(ds.ssh, f"field_{model.lower()}", "time", cmap="viridis", file_path=save_path)

#### SSH

In [ ]:
# ds.ssh.hvplot.image(
#     x="longitude",
#     y="latitude",
#     # groupby='time',
#     # rasterize=True,
#     width=500, height=400, cmap="viridis")

#### Gradient

In [ ]:
ds_obs["obs_p_n_grad"] = calculate_gradient(ds_obs["obs_p_n"], "Nx", "Ny", edge_order=1)

In [ ]:
calculate_gradient??

In [ ]:
# create_movie(ds.ssh_grad, f"field_{model.lower()}_grad", "time", cmap="Spectral_r", file_path=save_path)

In [ ]:
# ds_obs.obs_p_n_grad.hvplot.image(
#     x="Nx",
#     y="Ny",
#     # groupby='time',
#     # rasterize=True,
#     width=500,
#     height=400,
#     cmap="Spectral_r",
# )

#### (Norm) Laplacian

In [ ]:
ds_obs["obs_p_n_lap"] = calculate_laplacian(ds_obs["obs_p_n"], "Nx", "Ny", edge_order=1)

In [ ]:
# create_movie(ds.ssh_lap, f"field_{model.lower()}_lap", "time", cmap="RdBu_r", file_path=save_path)

In [ ]:
# ds_obs.obs_p_n_lap.hvplot.image(
#     x="Nx",
#     y="Ny",
#     # groupby='time',
#     # rasterize=True,
#     width=500,
#     height=400,
#     cmap="RdBu_r",
# )